In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import copy
path = "kabu.csv"
aa = pd.read_csv(path)
col = "nikkei225"#調べたい列名を入れる

d_t = 100 #トレンドの最小の大きさ
d_d = 1000 #新しいトレンドを作るerrorの閾値
d_e = 0#

g=[0,len(aa)-1]#trend change pointを入れていく
e=[0,len(aa)-1] #set of trends whose trend error is large

#firstからlastまでのデータでtrend change point探す
def eval():#gからeを作る
  trend = []
  for i in range(0,len(g)-1):
    katamuki = (aa[col][g[i+1]]-aa[col][g[i]])/(g[i+1]-g[i])
    error_sum=0
    for j in range(g[i],g[i+1]):
      estimated = aa[col][g[i]] + katamuki*(j-g[i])
      real = aa[col][j]
      error = abs(estimated-real)
      error_sum = error_sum + error
    te = error_sum/(g[i+1]-g[i])
    trend.append(te)
  d_e = sum(trend)/len(trend)
  for i in range(0,len(trend)):
    if trend[i] > d_e:
      if g[i] not in e:
        e.append(g[i])
      if g[i+1] not in e:
        e.append(g[i+1])
  e.sort()

def seg(first,last):#一つのトレンドを分割する関数
  if (last-first) < 2*d_t:
    return
  first_point = aa[col][first]
  last_point = aa[col][last]
  katamuki = (last_point-first_point)/(last-first)
  max_error = 0
  point=-1
  for i in range(first+d_t,last+1-d_t):
    real_value = aa[col][i]
    estimated_value = aa[col][first] + katamuki*(i-first)
    error = abs(real_value-estimated_value)
    if error > max_error:#一番大きいerrorとそのときのインデックスを求める
      max_error = error
      point = i  
  if point == -1:
    point = first+d_t
  if max_error < d_d:#一番大きなerrorが閾値より小さかったら分割はしない
    return 
    #ここからstag調査
  else:  
    h=[]
    for i in range(point-d_t,point+d_t):
      if abs(aa[col][i]-aa[col][point])<d_d/2:#pointに近いデータのインデックスをhに入れる
        h.append(i) 
    if (h[len(h)-1]-h[0])>d_t and (len(h)-1)>(h[len(h)-1]-h[0])/2:#point周りでstagあるとき
      if h[0] not in g:
        g.append(h[0])#stagの左端でトレンドを変化させる
      if point not in g:
        g.append(point)#pointでトレンドを変化させる
      if h[len(h)-1] not in g:
        g.append(h[len(h)-1])#stagの右端でトレンドを変化させる
      seg(first,h[0])#stagの左でもう一度分割
      seg(h[len(h)-1],last)#stagの右でもう一度分割
    else:#point周りでstagないとき
      if point not in g:
        g.append(point)#pointでトレンドを変化させる
      seg(first,point)#pointより左のところでもう一度分割
      seg(point,last)#pointより右のところでもう一度分割
  g.sort()      
#以下TBSM
count=0
while True:
  if count!=0:
    eval()
  count = count+1  
  g_old = copy.copy(g)
  for i in range(0,len(e)-1):
    if (e[i+1]-e[i]) < 2*d_t:
      continue
    else:
      d_max = 0
      for j in range(e[i]+d_t,e[i+1]-d_t):
        katamuki = (aa[col][e[i+1]]-aa[col][e[i]])/(e[i+1]-e[i])
        estimated = aa[col][e[i]] + katamuki*(j-e[i])
        real = aa[col][j]
        error = abs(estimated-real)
        if d_max < error:
          d_max = error
        d_d = d_max
      seg(e[i],e[i+1])
  if g_old == g:
    break     

e.sort()
print("trend change point")
print(e)
y=[]
for i in e:
  y.append(aa[col][i])
aa.plot()
plt.plot(e,y)  